# Import Libraries

In [0]:
import pydicom
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
import warnings
import random
import os, argparse
import importlib
import matplotlib.pyplot as plt

from imageio import imwrite as imsave
from skimage.io import imread
from skimage import exposure
from sklearn.metrics import roc_curve, auc, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from skimage.transform import resize as imresize
from skimage.transform import rotate

import tensorflow as tf
from keras import backend as K
sess = tf.Session()
K.set_session(sess)

from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.layers as layers
from keras.layers import Concatenate, Add, Flatten, Dense, Input
from keras.models import Model, load_model
from keras.optimizers import SGD, Nadam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

def normalize_element(x, m):
    Y = (x-m[0])/(m[1]-m[0])
    return Y  
vnormalize = np.vectorize(normalize_element, excluded=['m'])

# Load Data
Load the patient list and extract metadata. <br>
Studies taken before 2018 were used as test data and others were used for the model development. The development dataset was split 8:2 into training and validation sets.

In [0]:
# Load patient list
df_list0 = pd.read_excel('REVIEWLIST.xlsx', converters={
    'PatientID':str,
    'StudyDateTime':pd.to_datetime
})

df_list = df_list0[df_list0.check!=-1].copy()

pids = df_list['PatientID'].tolist()
dt = df_list['StudyDateTime'].tolist()
fx = df_list['FX'].tolist()
sL = df_list['StrongLabel'].tolist()
wL = df_list['WeakLabel'].tolist()
RL = df_list['R1L0'].tolist()

In [0]:
# Coding metadata
datasetpath = ##DATASET_PATH
ids_raw = np.array([a.split('_')[0] for a in os.listdir(datasetpath)])

ids_test = []
labels_test = []
ids_nontest = []
labels_nontest = []

def labeler(pid, time, f, slabel, wlabel, rl, n=0, mode='weak', testdate=datetime(2017,12,31)):
    date = datetime.strptime(str(time),"%Y-%m-%d %H:%M:%S")
    T = False
    if date > testdate: T = True

    ind = pid+str(n)
    if mode == 'weak':
        if wlabel == 0: return T, ind, wlabel
        elif rl == n: return T, ind, wlabel
        elif rl == 2: return T, ind, wlabel
        else: return T, ind, 0
    elif mode == 'strong':
        if slabel == 0: return T, ind, slabel
        elif rl == n: return T, ind, slabel
        elif rl == 2: return T, ind, slabel
        else: return T, ind, 0        

    return None

for pid, time, f, slabel, wlabel, rl in zip(pids, dt, fx, sL, wL, RL):
    date = datetime.strptime(str(time),"%Y-%m-%d %H:%M:%S")
    
    for n in range(2):
        ind = pid+str(n)
        if not ind in ids_raw: continue
        T, ID, L = labeler(pid, time, f, slabel, wlabel, rl, n=n, 
                           mode='strong',
                           testdate=datetime(2017,12,31))
        if T:
            ids_test.append(ID)
            labels_test.append(L)
        else:
            ids_nontest.append(ID)
            labels_nontest.append(L)

ids_test = np.array(ids_test)
labels_test = np.array(labels_test)
ids_nontest = np.array(ids_nontest)
labels_nontest = np.array(labels_nontest)

In [0]:
# Splitting train and validation set
valid_size = 0.2    ### portion out of nontest dataset

ids_train, ids_valid, labels_train, labels_valid = train_test_split(ids_nontest, labels_nontest, test_size=valid_size, stratify=labels_nontest)
print(len(ids_train), len(ids_valid), len(ids_test))

partition = {}
partition['train'] = (ids_train, labels_train)
partition['valid'] = (ids_valid, labels_valid)
partition['test'] = (ids_test, labels_test)

1012 254 258


# Model
A dual-input CNN-based deep neural network was constructed by merging two ResNet models in parallel.

In [0]:
def my_data_gen(dataset, datasetpath, _datagen, inputshape=(200,200), batch_size=4, num_classes = 2, gray_input=True):
    
    ids, labels = dataset[0], dataset[1]
    while True:
        batch_idx = np.random.choice(len(ids), batch_size)
        batch_ap = []
        batch_lat = []
        for ind in ids[batch_idx]:
            ap0 = imread(os.path.join(datasetpath, ind+'_0.png'))
            ap0 = imresize(ap0, inputshape, mode='reflect', anti_aliasing=True)
            ap0 = np.expand_dims(ap0, axis=-1)
            batch_ap.append(ap0)
            lat0 = imread(os.path.join(datasetpath, ind+'_1.png'))
            lat0 = imresize(lat0, inputshape, mode='reflect', anti_aliasing=True)
            lat0 = np.expand_dims(lat0, axis=-1)
            batch_lat.append(lat0)
        batch_ap = np.array(batch_ap)
        batch_lat = np.array(batch_lat)
        
        gap = _datagen.flow(batch_ap, batch_size=batch_size, shuffle=False)
        glat = _datagen.flow(batch_lat, batch_size=batch_size, shuffle=False)
        batch_x1 = gap.next()
        batch_x2 = glat.next()
        if gray_input:
            batch_x1 = np.concatenate((batch_x1,)*3, axis=-1)
            batch_x2 = np.concatenate((batch_x2,)*3, axis=-1)
        
        batch_y = np.array(labels[batch_idx])
        
        yield [batch_x1, batch_x2], to_categorical(batch_y, num_classes = num_classes)

def preprocess_adapthist(x):
    result = exposure.equalize_adapthist(x[:,:,0], clip_limit=0.03)
    result = np.expand_dims(result, axis=-1)
    if x.shape[2] == 3:
        result = np.concatenate((result,)*3, axis=-1)
    result = vnormalize(x=result, m=[np.min(result), np.max(result)])
    return result

def preprocess_normonly(x):
    return vnormalize(x=x, m=[np.min(x), np.max(x)])

data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     fill_mode='nearest',
                     preprocessing_function=preprocess_adapthist,
                    )

image_datagen = ImageDataGenerator(**data_gen_args)

In [0]:
################## model development ######################
input_shape = (200,200,3)
num_classes = 2
gray_input=True

batch_size=4
epochs = 25

my_data_gen_args = dict(batch_size=batch_size,
                        num_classes=num_classes,
                        gray_input=gray_input,
                        inputshape=input_shape[:2],
                       )

datasetpath = ##DATASET_PATH

In [0]:
wname = ##VERSION_NAME

result_dir = os.path.join('result', wname)
if not os.path.exists(result_dir):
    os.mkdir(result_dir)
result_dir_model = os.path.join(result_dir, 'model')
if not os.path.exists(result_dir_model):
    os.mkdir(result_dir_model)

model_path = os.path.join(result_dir_model, '{epoch:02d}-{val_loss:.6f}.hdf5')

In [0]:
# Save the partition
partition_path = os.path.join(result_dir, 'partition.pkl')
f = open(partition_path,"wb")
pickle.dump(partition,f)
f.close()

In [0]:
# Reset session
K.clear_session()
sess = tf.Session()
K.set_session(sess)

################ MODEL ##################
base_model1 = ResNet50(include_top=False,
                            weights='imagenet',
                            input_shape=input_shape)

base_model2 = ResNet50(include_top=False,
                            weights='imagenet',
                            input_shape=input_shape)

for layer in base_model1.layers:
    layer.name = layer.name+'_1'
for layer in base_model2.layers:
    layer.name = layer.name+'_2'

x1 = Flatten()(base_model1.output)
x2 = Flatten()(base_model2.output)
x = Concatenate()([x1, x2])

x = Dense(num_classes, activation='softmax', name='predictions')(x)

model = Model([base_model1.input,base_model2.input], x)

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_1 (InputLayer)          (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
input_2_2 (InputLayer)          (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad_1 (ZeroPadding2D)     (None, 206, 206, 3)  0           input_1_1[0][0]                  
__________________________________________________________________________________________________
conv1_pad_2 (ZeroPadding2D)     (None, 206, 206, 3)  0           input_2_2[0][0]                  
__________________________________________________________________________________________________
conv1_1 (C

# Train the Model

In [0]:
# Generators
train_generator = my_data_gen(partition['train'], datasetpath, image_datagen, **my_data_gen_args)
valid_generator = my_data_gen(partition['valid'], datasetpath, image_datagen, **my_data_gen_args)

# Start Fine-tuning
computing_time_start = datetime.now()

cb_checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                                verbose=1, save_best_only=True)
cb_early_stopping = EarlyStopping(monitor='val_loss', patience=10)

hist = model.fit_generator(generator=train_generator,
                           steps_per_epoch=np.ceil(5*partition['train'][0].size/batch_size),
                           validation_data=valid_generator,
                           validation_steps=np.ceil(5*partition['valid'][0].size/batch_size),
                           epochs=epochs,
                           verbose=1,
                           callbacks=[cb_checkpoint, cb_early_stopping],
                          )

# print computing time
computing_time_end = datetime.now()
computint_time_len = str(computing_time_end - computing_time_start)
print('COMPUTING TIME: ' + computint_time_len)

def save_loss_figure(hist, savepath, title=None, save=True, show=True):
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['acc'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='lower left')

    if title is not None:
        loss_ax.set_title(title)
    
    if save: plt.savefig(os.path.join(savepath, 'loss.png'))
    if show: plt.show()
    plt.close(fig)

save_loss_figure(hist, result_dir, title=computint_time_len, save=True, show=True)

# Test the Model

In [0]:
# Load the saved trained model
K.clear_session()
sess = tf.Session()
K.set_session(sess)

model = load_model('MODEL_PATH')
datasetpath = ##TEST_SET_PATH

In [0]:
# Load test data and predict
test_X1 = []
test_X2 = []

for ind in ids_valid:
    ap0 = imread(os.path.join(datasetpath, ind+'_0.png'))
    ap0 = imresize(ap0, input_shape[:2], mode='reflect', anti_aliasing=True)
    ap0 = np.expand_dims(ap0, axis=-1)
    ap0 = preprocess_adapthist(ap0)
    test_X1.append(ap0)
    lat0 = imread(os.path.join(datasetpath, ind+'_1.png'))
    lat0 = imresize(lat0, input_shape[:2], mode='reflect', anti_aliasing=True)
    lat0 = np.expand_dims(lat0, axis=-1)
    lat0 = preprocess_adapthist(lat0)
    test_X2.append(lat0)
test_X1 = np.array(test_X1)
test_X2 = np.array(test_X2)
if gray_input:
    test_X1 = np.concatenate((test_X1,)*3, axis=-1)
    test_X2 = np.concatenate((test_X2,)*3, axis=-1)

preds = model.predict([test_X1, test_X2])
Y_test = to_categorical(np.array(labels_valid), num_classes = num_classes)

In [0]:
# save test results
columns = ['IDS']
for i in range(preds.shape[1]):
    columns.append('pred'+str(i))
columns.append('truth')

test_results = np.column_stack((ids_test, preds, np.where(Y_test==1)[1]))
excelfilepath = os.path.join(result_dir, 'test_results.xlsx')
pd.DataFrame(test_results, columns=columns).to_excel(excelfilepath, index=False)

In [0]:
# ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve 1 (area = %0.2f)' % roc_auc[1])

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.axes().set_aspect('equal')
plt.savefig(os.path.join(result_dir,'ROC.png'))
plt.show()

# Code for class-activation map (CAM)
CAM was based on [GradCAM](https://github.com/jacobgil/keras-grad-cam). Sample result is not provided.

In [0]:
def generate_cam2(img_tensor, model, class_index, activation_layer):
    inp1, inp2 = model.input
    y_c = model.output.op.inputs[0][0, class_index]
    A_k = model.get_layer(activation_layer).output

    get_output = K.function([inp1, inp2], [A_k, K.gradients(y_c, A_k)[0], model.output])
    [conv_output, grad_val, model_output] = get_output([img_tensor[0], img_tensor[1]])

    conv_output = conv_output[0]
    grad_val = grad_val[0]

    weights = np.mean(grad_val, axis=(0, 1))

    grad_cam = np.zeros(dtype=np.float32, shape=conv_output.shape[0:2])
    for k, w in enumerate(weights):
        grad_cam += w * conv_output[:, :, k]

    grad_cam = np.maximum(grad_cam, 0)       
    grad_cam = imresize(grad_cam, img_tensor[0].shape[1:3])
    return grad_cam

In [0]:
N = ##
tid = ids_test[N]
tlabel = labels_test[N]

In [0]:
ap0 = imread(os.path.join(datasetpath, tid+'_0.png'))
ap0 = imresize(ap0, input_shape[:2], mode='reflect', anti_aliasing=True)
x1 = np.expand_dims(ap0, axis=-1)
x1 = preprocess_adapthist(x1)
lat0 = imread(os.path.join(datasetpath, tid+'_1.png'))
lat0 = imresize(lat0, input_shape[:2], mode='reflect', anti_aliasing=True)
x2 = np.expand_dims(lat0, axis=-1)
x2 = preprocess_adapthist(x2)

XX1 = np.expand_dims(x1, axis=0)
XX2 = np.expand_dims(x2, axis=0)
XX1 = np.concatenate((XX1,)*3, axis=-1)
XX2 = np.concatenate((XX2,)*3, axis=-1)
print(tid, tlabel)

pred = model.predict([XX1,XX2])
print(pred)

grad_cam1 = generate_cam2([XX1, XX2], model, int(tlabel), 'activation_49_1')
grad_cam2 = generate_cam2([XX1, XX2], model, int(tlabel), 'activation_98_2')

fig, axes = plt.subplots(3,2, figsize=(10,15))
ax = axes.ravel()

fig0 = ax[0].imshow(XX1[0][:,:,0], cmap=plt.cm.gray)
fig0 = ax[1].imshow(XX2[0][:,:,0], cmap=plt.cm.gray)
ax[0].set_title(tid)
ax[1].set_title(tlabel)

fig0 = ax[2].imshow(XX1[0][:,:,0])
fig0 = ax[2].imshow(grad_cam1, alpha=0.4, cmap=plt.cm.jet)
ax[2].set_title(pred)

fig0 = ax[3].imshow(XX2[0][:,:,0])
fig0 = ax[3].imshow(grad_cam2, alpha=0.4, cmap=plt.cm.jet)

ax[4].imshow(ap0, cmap='gray')
ax[5].imshow(lat0, cmap='gray')

plt.savefig(os.path.join(result_dir,'FN','{}_{}.png'.format(tid,tlabel)))
plt.show()
plt.close(fig)